In [1]:
# 高德地图：http://ditu.amap.com/
# 高德地图poi：http://lbs.amap.com/api/webservice/guide/api/search/#text
# 首先得申请一个下载poi数据的key，然后在对应位置填入你的key
# 关键字查询

import json
import xlwt
from datetime import datetime
from urllib import request
from urllib.parse import quote
import time
import os


# 获取数据
def get_data(pageindex, url_amap):
    global total_record
    # 暂停500毫秒，防止过快取不到数据
    time.sleep(0.5)
    print('解析页码： ' + str(pageindex) + ' ... ...')
    url = url_amap.replace('pageindex', str(pageindex))
    # 中文编码
    url = quote(url, safe='/:?&=')
    html = ""

    with request.urlopen(url) as f:
        html = f.read()
        rr = json.loads(html)
        if total_record == 0:
            total_record = rr['count']
        return rr['pois']


def getPOIdata(page_size, json_name, url_amap):
    global total_record
    print('获取POI数据开始')
    josn_data = get_data(1, url_amap)
    if (int(total_record) % page_size) != 0:
        page_number = int(int(total_record) / page_size) + 2
    else:
        page_number = int(int(total_record)/ page_size) + 1

    with open(json_name, 'w') as f:
        # 去除最后]
        f.write(json.dumps(josn_data).rstrip(']'))
        for each_page in range(2, page_number):
            html = json.dumps(get_data(each_page, url_amap)).lstrip('[').rstrip(']')
            if html:
                html = "," + html
            f.write(html)
            print('已保存到json文件：' + json_name)
        f.write(']')
    print('获取POI数据结束')

# 写入数据到excel
def write_data_to_excel(json_name, hkeys, bkeys, name):
    # 获取当前日期
    today = datetime.today()
    # 将获取到的datetime对象仅取日期
    today_date = datetime.date(today)

    # 从文件中读取数据
    fp = open(json_name, 'r')
    result = json.loads(fp.read())
    # 实例化一个Workbook()对象(即excel文件)
    wbk = xlwt.Workbook()
    # 新建一个名为Sheet1的excel sheet。此处的cell_overwrite_ok =True是为了能对同一个单元格重复操作。
    sheet = wbk.add_sheet('Sheet1', cell_overwrite_ok=True)

    # 创建表头
    # for循环访问并获取数组下标enumerate函数
    for index, hkey in enumerate(hkeys):
        sheet.write(0, index, hkey)

    # 遍历result中的每个元素。
    for i in range(len(result)):
        values = result[i]
        n = i + 1
        for index, key in enumerate(bkeys):
            val = ""
            # 判断是否存在属性key
            if key in values.keys():
                val = values[key]
            sheet.write(n, index, val)
   
    # wbk.save(name + str(today_date) + '.xls')
    wbk.save(name +'.xls')
    # print('保存到excel文件： ' + name + str(today_date) + '.xls!')
    print('保存到excel文件： ' + name + '.xls!')


if __name__ == '__main__':
    json_name = 'data_amap.json'
    #os.makedirs('data_index')
    # 高德地图poi：http://lbs.amap.com/api/webservice/guide/api/search/#text
    # city = ["北京市","天津市","上海市","重庆市","河北省","山西省","辽宁省","吉林省","黑龙江","江苏省","浙江省",
    #        "安徽省","福建省","江西省","山东省","河南省","湖北省","湖南省","广东省","海南省","四川省","贵州省",
    #        "云南省","陕西省","甘肃省","青海省","台湾省","内蒙古自治区","广西壮族自治区","西藏自治区","宁夏回族自治区","新疆维吾尔自治区","香港特别行政区","澳门特别行政区"]

    # 解析city.json数据，读取城市列表
    city = ["310113"]
    # city = ["嘉定区","金山区","闵行区","浦东新区","青浦区","松江区"]
    # data = open("city.json", encoding="utf-8-sig")
    # # 转换为python对象
    # strJson = json.load(data)
    # for i in range(len(strJson)):
    #     city.append(strJson[i]['n'])
    # keyword = ["银行"]
    # 体育休闲服务 公共设施 商店  饭店   银行
    # 关键词："加油站","汽车销售","汽车维修","美食","购物","生活服务","体育休闲","医疗保健","宾馆酒店","风景",...等等
    # type具体可以查表http://lbs.amap.com/api/webservice/guide/api/search/#text
    # type = ["010000","020000","030000","050000","060000","070000","080000","090000","100000","110000","140000","150000","160000"]
    # type = ["120300",'120301','120302','120303','120304']
    # type = ["体育休闲服务","购物服务","金融保险服务","公共设施","交通设施服务"]
    type = ["交通设施服务"]

    # 读出来的xls文件的坐标系为GCJ-02 需要转换

    key ='61c256acd1b853e34a6b9a31033c399d'

    for i in range(0, len(city)):

        # for j in range(0, len(keyword)):
        for j in range(0, len(type)):

            # url_amap = 'http://restapi.amap.com/v3/place/text?key=e7d3e7cf12933eb1a8d0e48117cc5bcc&keywords=' + keyword[j] + '&types=' + type[j] + '&city=' + city[i] + '&citylimit=true&children=1&offset=20&page=pageindex&extensions=all'
            # url_amap = 'http://restapi.amap.com/v3/place/text?key='+key+'&keywords=' + keyword[j] + '&city=' + city[i] + '&citylimit=true&children=1&offset=20&page=pageindex&extensions=all'
            url_amap = 'http://restapi.amap.com/v3/place/text?key='+key+'&types=' + type[j] + '&city=' + city[i] + '&citylimit=true&children=1&offset=20&page=pageindex&extensions=all'
            page_size = 25  # 每页记录数据，强烈建议不超过25，若超过25可能造成访问报错
            page_index = r'page=1'  # 显示页码
            global total_record
            total_record = 0
            # Excel表头
            # hkeys = ['id', '行业类型', '名称', '类型', '地址', '联系电话', 'location', '省份代码', '省份名称', '城市代码', '城市名称', '区域代码', '区域名称',
            #         '所在商圈']
            hkeys = ['id', '行业类型', '名称', '类型', '地址', '联系电话', 'location', '省份代码', '省份名称', '城市代码', '城市名称', '区域代码', '区域名称',
                    '所在商圈']
            # 获取数据列
            bkeys = ['id', 'biz_type', 'name', 'type', 'address', 'tel', 'location' 'pcode', 'pname', 'citycode',
                    'cityname','adcode', 'adname', 'business_area']
            # 写入数据到json文件，第二次运行可注释
            getPOIdata(page_size, json_name, url_amap)
            # 读取json文件数据写入到excel
            # os.makedirs("data_index\\"+city[i])
            write_data_to_excel(json_name, hkeys, bkeys, "data_output\\" + city[i] + type[j] + "-高德地图")
            if (i % 13 == 0):
               time.sleep(15)
            elif (i % 13 != 0):
               time.sleep(15)

获取POI数据开始
解析页码： 1 ... ...
解析页码： 2 ... ...
已保存到json文件：data_amap.json
解析页码： 3 ... ...
已保存到json文件：data_amap.json
解析页码： 4 ... ...
已保存到json文件：data_amap.json
解析页码： 5 ... ...
已保存到json文件：data_amap.json
解析页码： 6 ... ...
已保存到json文件：data_amap.json
解析页码： 7 ... ...
已保存到json文件：data_amap.json
解析页码： 8 ... ...
已保存到json文件：data_amap.json
解析页码： 9 ... ...
已保存到json文件：data_amap.json
解析页码： 10 ... ...
已保存到json文件：data_amap.json
解析页码： 11 ... ...
已保存到json文件：data_amap.json
解析页码： 12 ... ...
已保存到json文件：data_amap.json
解析页码： 13 ... ...
已保存到json文件：data_amap.json
解析页码： 14 ... ...
已保存到json文件：data_amap.json
解析页码： 15 ... ...
已保存到json文件：data_amap.json
解析页码： 16 ... ...
已保存到json文件：data_amap.json
解析页码： 17 ... ...
已保存到json文件：data_amap.json
解析页码： 18 ... ...
已保存到json文件：data_amap.json
获取POI数据结束
保存到excel文件： data_output\310113交通设施服务-高德地图.xls!


In [2]:
len(keyword)

5

In [ ]:
len(type)

In [4]:
len(type)

1

In [ ]:
len(city)

In [3]:
import pandas as pd
import transbigdata as tbd
import geopandas as gpd

# name='交通设施服务'
# name='金融保险服务'
# name='购物服务'
name='公共设施'

# json=gpd.read_file('data_output\SHP\奉贤区+公园\徐汇区.shp')
json=gpd.read_file('data_input\区划\崇明区.json')
data=pd.read_csv('data_output\崇明区{}.csv'.format(name))
data=tbd.clean_outofshape(data,json,col=['lng','lat'])
data['lng'],data['lat']=tbd.gcj02towgs84(data['lng'],data['lat'])
data.drop_duplicates(subset=['名称'],inplace=True)
data.to_csv('data_output\崇明区{}_wgs84.csv'.format(name),encoding='utf-8-sig')


In [4]:
# 去除重复
import pandas as pd

name='公园内部设施'

data=pd.read_csv('徐汇区{}_wgs84.csv'.format(name))
data.drop_duplicates(subset=['名称'],inplace=True)
data.to_csv('徐汇区{}_wgs84.csv'.format(name),index=False,encoding='utf-8_sig')

In [ ]:
# 法1：用网页可视化  作者的网页：https://ni1o1.github.io/ODview/build/ （将OD_data.csv导入即可 可能有点卡）
# 法2：高德地图

import webbrowser
import os
import pandas as pd

# 创建 HTML 文件
html_content = f"""
<!doctype html>
<html>
<head>
   <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="initial-scale=1.0, user-scalable=no, width=device-width">
    <title>OD数据展示</title>
    <link rel="stylesheet" href="https://a.amap.com/jsapi_demos/static/demo-center/css/demo-center.css"/>
    <script type="text/javascript" src="https://webapi.amap.com/maps?v=2.0&key=c9a99c0fe85b87c0a023b871978d00dc"></script>
    <style>
        html, body,
        #container {{
            height: 100%;
            width: 100%;
            margin: 0;
        }}
    </style>
</head>
<body>
    <div id="container"></div>

    <script type="text/javascript">
        var map = new AMap.Map('container', {{
            zoom: 12,
            viewMode:'3D',
            center: [114.0214, 22.603918],
            showIndoorMap: false,
        }});

         AMap.plugin(
            [
                'AMap.ToolBar',
                'AMap.Scale',
                'AMap.MapType',
                'AMap.Geolocation',
                'AMap.ControlBar'
            ],
            function() {{
                map.addControl(new AMap.ToolBar({{
                    position: {{
                        top: '100px',
                        left: '20px'
                    }}
                }}));
                map.addControl(new AMap.Scale());
                map.addControl(new AMap.MapType());
                map.addControl(new AMap.Geolocation({{
                    position: {{
                        top: '180px',
                        left: '20px'
                    }}
                }}));
                map.addControl(new AMap.ControlBar({{
                    position: {{
                        top: '1px',
                        left: '1px'
                    }}
                }}));
            }})

        map.on('complete', function () {{
            var layer = new AMap.LabelsLayer({{
                zooms: [3, 20],
                zIndex: 1000,
                collision: false
            }});

            map.add(layer);

            var markers = [];
            var positions = {data[['lng', 'lat']].to_dict(orient='records')};
            AMap.convertFrom(positions, 'positions', function (status, result) {{
                if (result.info === 'ok') {{
                var lnglats = result.locations;
                    }}
                }});
            console.log(positions)
            console.log(positions[0].slon)

            var icon = {{
                type: 'image',
                image: 'https://webapi.amap.com/theme/v1.3/markers/n/mark_b.png',
                size: [6, 9],
                anchor: 'bottom-center',
            }};

            for (var i = 0; i < positions.length; i++) {{

                var curPosition = positions[i];
                var curData = {{
                    position: [curPosition.slon, curPosition.slat],
                    icon
                }};

                var labelMarker = new AMap.LabelMarker(curData);
                markers.push(labelMarker);
            }}

            layer.add(markers);
        }});
    </script>
</body>
</html>
"""

html_file_path = "data_output/OD_map.html"
with open(html_file_path, "w", encoding="utf-8") as html_file:
    html_file.write(html_content)

file_url = "file://" + os.path.abspath(html_file_path)
webbrowser.open(file_url)


# 可以在此处添加等待，以保持程序运行状态，或者将其保存为一个独立的脚本并在终端中运行
# input("Press Enter to exit...")


# 在上述代码中，DBSCAN 是密度聚类算法，eps 和 min_samples 是DBSCAN算法的两个关键参数，它们的作用如下：
#
# eps（epsilon）：是指定点之间的最大距离，用于确定邻域的大小。如果两个点的距离小于 eps，则认为它们是邻居关系。这个参数影响着聚类的密度，即在 eps 范围内有足够多的点才能形成一个聚类。
#
# min_samples：是指在一个点的邻域内至少需要有多少个点，包括该点自身，才能形成一个簇。这个参数用于控制簇的最小样本数，可以防止将噪声点单独作为一个簇。
#
# 在这里，DBSCAN(eps=0.005, min_samples=2) 表示使用 DBSCAN 算法进行聚类，设置邻域最大距离 eps 为 0.005，最小样本数 min_samples 为 2。这样的设置通常需要根据具体的数据特点来调整，可以通过尝试不同的参数值来获得最佳的聚类效果。
#
# 在聚类完成后，fit_predict 方法返回每个样本点的聚类标签。聚类标签为正数表示该点属于某个簇，为 -1 表示该点为噪声点。
#
# 在脚本中，聚类结果被添加到DataFrame中的'Cluster'列中，然后在地图上显示每个点的聚类情况。密度较大的点将具有相同的聚类标签，以便更好地可视化地展示在地图上。不同的聚类使用不同颜色的图标显示。
